In [ ]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
folder =  '/home/lab/Documents/all_vids/selected_vids/14_07_2023_CS_2' # change this to your folder with the selected videos

In [ ]:
# get all the paths of the videos in the folder
vid_paths = []
for vid in os.listdir(folder):
    vid = os.path.join(folder, vid)
    if vid.endswith('.mov'):
            vid_paths.append(vid)

In [ ]:
def find_contours(input_file):

    video_capture = cv2.VideoCapture(input_file)
    
    # Initialize variables
    contour_diff_list = []

    prev_frame = None
    while True:
        ret, frame = video_capture.read()
        if not ret:
            break

        # converts the current frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # applies a Gaussian blur to the grayscale frame, helps reduce noise and minor variations in pixel values 
        gray = cv2.GaussianBlur(gray, (21, 21), 0)

        # Calculate contour difference using cv2.absdiff
        if prev_frame is not None:
            # difference betweeen the two frames
            frame_delta = cv2.absdiff(prev_frame, gray)
            # turns those parts about the theshold (30) t0 255 (white)
            thresh = cv2.threshold(frame_delta, 30, 255, cv2.THRESH_BINARY)[1]
            # morphological operation to dilate, this enhances the motion regions
            thresh = cv2.dilate(thresh, None, iterations=2)
            contours, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # cv2.RETR_EXTERNAL

            # If no difference add 0, if multiple find the max and add it
            if len(contours) == 0:
                contour_diff_list.append(0)
            else:
                tmp_contours = []
                for contour in contours:
                    tmp_contours.append(cv2.contourArea(contour))
                contour_diff_list.append(np.max(tmp_contours))


        # Set the current frame as the previous frame for the next iteration
        prev_frame = gray

    # Release video capture and writers
    video_capture.release()

    return np.array(contour_diff_list)

In [ ]:
# a function to plot the contour array
def contour_plot(array, fps):
    t_list = np.array((range(0, len(array)))) / fps  # can
    ax.plot(t_list, array)

In [ ]:
# Plots the first 20 files contour arrays, change to however many you want
for i in range(20):
    fig, ax = plt.subplots(figsize=(20, 6), layout='constrained')
    contour_plot(find_contours(vid_paths[i]), 38)
    plt.show()